In [1]:
import src.acquire
import src.prepare
import src.explore
import src.model

In [2]:
path='zillow_df.csv'

df, X_train_explore, \
    X_train_scaled, y_train, \
    X_validate_scaled, y_validate, \
    X_test_scaled, y_test = src.prepare.wrangle_zillow(path)

X_train_explore.shape, X_validate_scaled.shape, X_test_scaled.shape

((11666, 18), (5000, 15), (4167, 15))

In [3]:
X_train_explore

,Unnamed: 0,bathroomcnt,bedroomcnt,calculatedfinishedsquarefeet,fireplace,garage,hottub_spa,lotsizesquarefeet,poolcnt,roomcnt,yearbuilt,id.1,zip,useid,year,zip_cc,useid_cc,year_cc
18116,18116,2.0,3.0,1941.0,0.0,1.0,0.0,10100.0,0.0,7.0,1962.0,60153,97067.0,261.0,1962.0,348,4,74
10677,10677,5.0,4.0,3583.0,0.0,0.0,0.0,16708.0,1.0,0.0,1964.0,52698,96030.0,261.0,1964.0,46,4,76
20113,20113,1.0,2.0,810.0,0.0,0.0,0.0,5000.0,0.0,0.0,1944.0,62152,95983.0,261.0,1944.0,1,4,56
6955,6955,2.0,2.0,1078.0,0.0,0.0,0.0,843582.0,0.0,0.0,1980.0,48970,96374.0,266.0,1980.0,207,8,92
10756,10756,3.0,3.0,1374.0,1.0,2.0,0.0,3920.0,0.0,5.0,1970.0,52777,97109.0,265.0,1970.0,366,7,82
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9033,9033,3.0,4.0,3340.0,0.0,2.0,0.0,9375.0,0.0,0.0,1990.0,51050,96987.0,261.0,1990.0,309,4,102
4244,4244,3.0,4.0,2582.0,0.0,0.0,0.0,7190.0,0.0,0.0,1990.0,46255,97329.0,261.0,1990.0,379,4,102
1434,1434,1.5,2.0,1020.0,0.0,1.0,0.0,0.0,0.0,5.0,1974.0,43440,96962.0,266.0,1974.0,290,8,86
17705,17705,3.0,2.0,1800.0,0.0,0.0,0.0,36886.0,1.0,0.0,1980.0,59741,96005.0,266.0,1980.0,22,8,92


In [4]:
# import pandas as pd
# df["zip_cc"] = df["zip"].cat.codes
# df

## Notes

1. I already uploaded my env.py file so need need to upload yours.  We only need one to access the data.


2. I already uploaded a .gitignore so also no worries there


3. Data is ready to go, i want to engeneer a feature that combines bed and bath, but get a weird error when I try.    we can trouble shoot after mvp, but code can be found in .prepare file


4. I want to leave 2 categorical variables after transforming them to numeric but I also need help with that
   - "regionidzip"
   - "propertylandusetypeid"
   - They need to be made in to categories before we can do any testing on them (Cat to Numeric)


5. id.1 and Unmaed:0 showd up we also need to delete them